In [1]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import datetime
import math, time
import itertools
from sklearn import preprocessing
import datetime
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from math import sqrt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
import yfinance as yf

Using TensorFlow backend.


In [2]:
def get_stock_data(stock_name):
    msft = yf.Ticker(stock_name)
    df=pd.DataFrame(msft.history(period="max"))
    df.reset_index(inplace=True)
    col_names = ['Date','Open','High','Close']
    df=df[col_names]
    df.sort_values(by=['Date'], inplace=True, ascending=False)
    return df

In [3]:
stock_name = 'GOOGL'
df = get_stock_data(stock_name)
df.head()

,Date,Open,High,Close
3928,2020-03-27,1127.47,1151.05,1110.26
3927,2020-03-26,1114.72,1171.48,1162.92
3926,2020-03-25,1124.58,1146.17,1101.62
3925,2020-03-24,1103.98,1133.33,1130.01
3924,2020-03-23,1056.37,1066.91,1054.13


In [4]:
df.drop('Date', axis=1, inplace=True)

In [5]:
def load_data(stock, seq_len):
    amount_of_features = len(stock.columns)
    data = stock.as_matrix() #pd.DataFrame(stock)
    sequence_length = seq_len + 1
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index: index + sequence_length])

    result = np.array(result)
    row = round(0.9 * result.shape[0])
    train = result[:int(row), :]
    x_train = train[:, :-1]
    y_train = train[:, -1][:,-1]
    x_test = result[int(row):, :-1]
    y_test = result[int(row):, -1][:,-1]

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], amount_of_features))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], amount_of_features))  

    return [x_train, y_train, x_test, y_test]

In [10]:
#Build the model
def build_model(layers):
    model = Sequential()

    model.add(LSTM(
        input_dim=layers[0],
        output_dim=layers[1],
        return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(
        layers[2],
        return_sequences=False))
    model.add(Dropout(0.2))

    model.add(Dense(
        output_dim=layers[2]))
    model.add(Activation("linear"))

    start = time.time()
    model.compile(loss="mse", optimizer="rmsprop",metrics=['accuracy'])
    print("Compilation Time : ", time.time() - start)
    return model

def build_model2(layers):
        d = 0.2
        model = Sequential()
        model.add(LSTM(128, input_shape=(layers[1], layers[0]), return_sequences=True))
        model.add(Dropout(d))
        model.add(LSTM(64, input_shape=(layers[1], layers[0]), return_sequences=False))
        model.add(Dropout(d))
        model.add(Dense(12, input_dim=8, kernel_initializer ='uniform', activation='relu'))
        model.add(Dense(8, kernel_initializer ='uniform', activation='relu'))
        model.add(Dense(1, kernel_initializer ='uniform', activation='sigmoid'))
        model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
        return model

In [7]:
window = 44
X_train, y_train, X_test, y_test = load_data(df[::-1], window)
print("X_train", X_train.shape)
print("y_train", y_train.shape)
print("X_test", X_test.shape)
print("y_test", y_test.shape)

X_train (3496, 44, 3)
y_train (3496,)
X_test (388, 44, 3)
y_test (388,)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:

# model = build_model([3,lag,1])
model = build_model2([3,window,1])

In [ ]:

model.fit(
    X_train,
    y_train,
    batch_size=5,
    epochs=100,
    validation_split=0.3,
    verbose=1)

Train on 2447 samples, validate on 1049 samples
Epoch 1/100
2447/2447 [==============================] - 22s 9ms/step - loss: 89321.0690 - accuracy: 0.0000e+00 - val_loss: 695690.5858 - val_accuracy: 0.0000e+00
Epoch 2/100
2447/2447 [==============================] - 21s 9ms/step - loss: 89285.4289 - accuracy: 0.0000e+00 - val_loss: 695690.5114 - val_accuracy: 0.0000e+00
Epoch 3/100
2447/2447 [==============================] - 21s 9ms/step - loss: 89285.4152 - accuracy: 0.0000e+00 - val_loss: 695690.5114 - val_accuracy: 0.0000e+00
Epoch 4/100
2447/2447 [==============================] - 25s 10ms/step - loss: 89285.4149 - accuracy: 0.0000e+00 - val_loss: 695690.5114 - val_accuracy: 0.0000e+00
Epoch 5/100
2447/2447 [==============================] - 22s 9ms/step - loss: 89285.4148 - accuracy: 0.0000e+00 - val_loss: 695690.5114 - val_accuracy: 0.0000e+00
Epoch 6/100
2447/2447 [==============================] - 24s 10ms/step - loss: 89285.4150 - accuracy: 0.0000e+00 - val_loss: 695690.5114

In [ ]:
trainScore = model.evaluate(X_train, y_train, verbose=0)
print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore[0], math.sqrt(trainScore[0])))

testScore = model.evaluate(X_test, y_test, verbose=0)
print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore[0], math.sqrt(testScore[0])))

In [ ]:
p = model.predict(X_test)
import matplotlib.pyplot as plt2

plt2.plot(p,color='red', label='prediction')
plt2.plot(y_test,color='blue', label='y_test')
plt2.legend(loc='upper left')
plt2.show()